<a href="https://colab.research.google.com/github/ahaanchaudhuri/ansiblehealthassessment/blob/main/convert_markdown_to_google_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Authenticate with Google Docs API
def authenticate_google_docs():
    SCOPES = ['https://www.googleapis.com/auth/documents']
    creds = None

    # Check if token.pickle exists
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or are invalid, log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_console()  # Use console flow for Colab

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return creds


# Parse Markdown Notes to a Structured Format
def parse_markdown_to_structure(markdown_notes):
    structured_notes = []
    lines = markdown_notes.splitlines()

    for line in lines:
        if line.startswith('# '):
            structured_notes.append({'type': 'heading1', 'content': line[2:]})
        elif line.startswith('## '):
            structured_notes.append({'type': 'heading2', 'content': line[3:]})
        elif line.startswith('### '):
            structured_notes.append({'type': 'heading3', 'content': line[4:]})
        elif line.startswith('- [ ] '):
            structured_notes.append({'type': 'checkbox', 'content': line[6:]})
        elif line.startswith('- '):
            structured_notes.append({'type': 'bullet', 'content': line[2:]})
        else:
            structured_notes.append({'type': 'text', 'content': line})

    return structured_notes

# Create and Format Google Doc
def create_google_doc(creds, structured_notes):
    service = build('docs', 'v1', credentials=creds)
    document = service.documents().create(body={"title": "Product Team Sync"}).execute()
    document_id = document.get('documentId')

    requests = []

    for note in structured_notes:
        if note['type'] == 'heading1':
            requests.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': note['content'] + '\n'
                }
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': 1,
                        'endIndex': len(note['content']) + 2
                    },
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_1'
                    },
                    'fields': 'namedStyleType'
                }
            })
        elif note['type'] == 'heading2':
            requests.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': note['content'] + '\n'
                }
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': 1,
                        'endIndex': len(note['content']) + 2
                    },
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_2'
                    },
                    'fields': 'namedStyleType'
                }
            })
        elif note['type'] == 'checkbox':
            requests.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': '\u2611 ' + note['content'] + '\n'
                }
            })
        elif note['type'] == 'bullet':
            requests.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': '\u2022 ' + note['content'] + '\n'
                }
            })

    service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()
    return document_id

# Step 4: Main Function
def main():
    markdown_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda
### 1. Sprint Review
- Completed Features
    - User authentication flow
    - Dashboard redesign
    - Performance optimization
        - Reduced load time by 40%
        - Implemented caching solution
- Pending Items
    - Mobile responsive fixes
    - Beta testing feedback integration

### 2. Current Challenges
- Resource constraints in QA team
- Third-party API integration delays
- User feedback on new UI
    - Navigation confusion
    - Color contrast issues

### 3. Next Sprint Planning
- Priority Features
    - Payment gateway integration
    - User profile enhancement
    - Analytics dashboard
- Technical Debt
    - Code refactoring
    - Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Notes
- Next sync scheduled for May 22, 2023
- Platform demo for stakeholders on May 25
- Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

    try:
        creds = authenticate_google_docs()
        structured_notes = parse_markdown_to_structure(markdown_notes)
        document_id = create_google_doc(creds, structured_notes)
        print(f"Google Doc created successfully! Document ID: {document_id}")
    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()

AttributeError: 'InstalledAppFlow' object has no attribute 'run_console'